<a href="https://colab.research.google.com/github/AlirezaPNouri/BERTEmbedding/blob/main/AStarSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing A star Search algorith to find relevant documents in a database

In [10]:
!pip install transformers
!pip install wget
!pip install statistics

     |████████████████████████████████| 3.8 MB 4.0 MB/s 
     |████████████████████████████████| 67 kB 3.8 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 6.5 MB 35.0 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=40768f423ed728d7408ee7e6e27197dc547cc776fc64e43d70ac36cefa0ae5ca
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7454 sha256=cd6106f24165ae8a43a39bcb354d09c480d1427dbcb484dcb93bed430eb6f4aa
  Stored in directory: /root/.cache/pip/wheels/37/09/e1/49ee45c0ce248a6e9c986aae82d32bbcba09c9f3b9d0c3406a
Successfully built stati

In [11]:
# libraries that are needed to import
import os
import time
import datetime
import sys
import random
import requests
import pickle
import string
import pandas as pd
import numpy as np
from numpy import median, array, exp
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.font_manager import list_fonts
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import seaborn as sns
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer

In [12]:
# All configs are here
MAX_DATA = 1000 # the maximum data that is extracted from the dataset
max_length = 512 # maximum number of tokens in a document (Bert input)
batch_size = 16 # For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
epochs = 25 # Number of training epochs (authors recommend between 2 and 4)
seed_val = 42 # Set the seed value all over the place to make this reproducible.
Max_Feature_Size = 20  ## ignore the features after this number
ratio_validation_size = 0.1
number_of_test_data = int(ratio_validation_size * MAX_DATA ) # number of data in testing phase
stopwords = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'of', 'within']
jump = 3 # jump size for gradient decent
range_n_clusters = [2, 3, 4, 5, 6] #number of different clustering to compare the performance of the model over different clustering
doc_num = 18 # document number for experiments
threshold_cluster_overlapping = 0.8 # the after jaccard index that we use to consider two clusters are overlapping 
doc_list = [item for item in range(number_of_test_data)]
 # list of documents' id to extract features
thres_doc = 0.01 # the similarity threshold between each cluster in two different documents


In [13]:
# Function to remove stopwords
def stop_word_remover(list1, stopwords):
  new_list = list()
  for word in list1:
    if word not in stopwords:
      new_list.append(word)
  return new_list
############################################################################  
#function to get unique values
def unique(list1):
    # initialize a null list
    unique_list = []
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return len(unique_list)
############################################################################  
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
############################################################################
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
############################################################################
#Function to find similarities
def find_similar(embedding_layers, word_index, original_sentence):
  score_dic = dict()
  # print(len(original_sentence))
  for j in range(1, min(256, len(original_sentence))):
    score_dic[original_sentence[j -1]] =  cosine_similarity([embedding_layers[word_index].to('cpu').numpy()], [embedding_layers[j].to('cpu').numpy()])[0][0]
    # print('word is {}, score is {}'.format(original_sentence[j -1], cosine_similarity([embedding_layers[word_index].to('cpu').numpy()], [embedding_layers[j].to('cpu').numpy()])[0][0]))
  return score_dic
############################################################################
# show th t-SNE for the tokens
def tsne_plot(sentences, output):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word, embed in zip(sentences, output):
        tokens.append(embed)
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
############################################################################
# Jaccard index function
def jaccard_similarity(text1_list, text2_list):
  a = set(text1_list)
  b = set(text2_list)
  return len(a.intersection(b))/len(a.union(b))
############################################################################
#function to normalize a vector
def normalizer_value(list_):
    temp_list = list()
    for vec in list_[2]:
        value = int(99* ((vec[1] - list_[1]) / (list_[0] - list_[1])) + 1)
        temp_list.append([vec[0],value])
    return temp_list
##########################################################################
#sigmoid function
def sigmoid(vec_):
  z = array(vec_)
  z = z/np.linalg.norm(z)
  g = 1 / (1 + exp(-z))
  return g
#######################################################################
#find similar documents
def similar_doc(doc_one, doc_two, thres_doc):
  list_sim = list()
  for clu_1 in doc_one:
    new_doc_one = list()
    for d1, f1 in clu_1.items():
      for fre in range(f1):
        new_doc_one.append(d1)
    total_jac = 0
    for clu_2 in doc_two:
      new_doc_two = list()
      for d2, f2 in clu_2.items():
        for fre in range(f2):
          new_doc_two.append(d2)
      jac_val = jaccard_similarity(new_doc_one, new_doc_two)
      # print(jac_val)
      # print(new_doc_one)
      # print(new_doc_two)
      # print('***********')
      if jac_val > thres_doc:
        total_jac += jac_val
    # total_jac /= len(doc_two)
    list_sim.append(total_jac)
  return list_sim
######################################################################
########softmax######################################################
#apply softmax over the scores
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [3]:
# # load picles' objects
import pickle

# pickle.dump(df, open("df.p", "wb"))
# pickle.dump(last_layer, open("last_layer.p", "wb"))
# pickle.dump(sentences, open("sentences.p", "wb"))
# pickle.dump(total_clustering_w_frequency, open("total_clustering_w_frequency.p", "wb"))
# pickle.dump(total_doc_word_groups, open("total_doc_word_groups.p", "wb"))
df = pickle.load(open("df.p", "rb"))
last_layer = pickle.load(open("last_layer.p", "rb"))
sentences = pickle.load(open("sentences.p", "rb"))
total_clustering_w_frequency = pickle.load(open("total_clustering_w_frequency.p", "rb"))
total_doc_word_groups = pickle.load(open("total_doc_word_groups.p", "rb"))

In [83]:
from collections import deque

class Graph:

    def __init__(self, adjacency_list, embedding_documents, starting_node):
        self.adjacency_list = adjacency_list
        self.embedding_documents = embedding_documents
        self.H = dict()
        for d_ in self.adjacency_list.keys():
          self.H[d_] = 51 - int(50*sum(abs(cosine_similarity([doc_embedding[d_].cpu().numpy()], [doc_embedding[starting_node].cpu().numpy()])[0])))
        print(self.H)
        
    # def get_neighbors(self, v):
    #     return self.adjacency_list[v]

    # # heuristic function with equal values for all nodes
    # def h(self, n):
    #   # H = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1}
    #   return self.H[n]

    def a_star_algorithm(self, start_node, stop_node):
        # open_list is a list of nodes which have been visited, but who's neighbors
        # haven't all been inspected, starts off with the start node
        # closed_list is a list of nodes which have been visited
        # and who's neighbors have been inspected
        open_list = set([start_node])
        closed_list = set([])

        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}

        g[start_node] = 0

        # parents contains an adjacency map of all nodes
        parents = {}
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None

            # find a node with the lowest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.H[v] < g[n] + self.H[n]:
                    n = v;

            if n == None:
                print('Path does not exist!')
                return None

            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if n == stop_node:
                reconst_path = []

                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]

                reconst_path.append(start_node)

                reconst_path.reverse()

                print('Path found: {}'.format(reconst_path))
                return reconst_path

            # for all neighbors of the current node do
            for (m, weight) in self.adjacency_list[n]:
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight

                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None

In [32]:
print(last_layer.shape)


torch.Size([100, 512, 768])
torch.Size([100, 768])


In [66]:
doc_matrix = dict()
doc_embedding = last_layer[:,0,:]
for first_doc in doc_list:
  temp_dict = list()
  for second_doc in doc_list:
    if first_doc != second_doc:
      temp_dict.append((second_doc, int(100*sum(abs(cosine_similarity([doc_embedding[first_doc].cpu().numpy()], [doc_embedding[second_doc].cpu().numpy()])[0])))))
      # if sum(similar_doc(total_clustering_w_frequency[first_doc], total_clustering_w_frequency[second_doc], thres_doc))>0.5:
      #   temp_dict.append((second_doc, int(100*sum(cosine_similarity([doc_embedding[first_doc].cpu().numpy()], [doc_embedding[second_doc].cpu().numpy()])[0]))))
  doc_matrix[first_doc] = temp_dict
print(doc_matrix)


# adjacency_list = doc_matrix
adjacency_list = dict()
list_added_nodes = list()
for doc_ in doc_matrix.keys():
  temp_list = list()
  for related_doc in doc_matrix[doc_]:
    if [doc_ , related_doc[0]] not in list_added_nodes and [related_doc[0], doc_] not in list_added_nodes:
      list_added_nodes.append([doc_ , related_doc[0]])
      temp_list.append(related_doc)
  adjacency_list[doc_] = temp_list


{0: [(1, 49), (2, 32), (3, 43), (4, 31), (5, 37), (6, 24), (7, 30), (8, 31), (9, 34), (10, 56), (11, 30), (12, 48), (13, 34), (14, 31), (15, 44), (16, 34), (17, 38), (18, 44), (19, 5), (20, 43), (21, 32), (22, 44), (23, 34), (24, 34), (25, 37), (26, 28), (27, 42), (28, 49), (29, 52), (30, 34), (31, 48), (32, 38), (33, 36), (34, 43), (35, 38), (36, 32), (37, 20), (38, 33), (39, 39), (40, 6), (41, 45), (42, 15), (43, 37), (44, 48), (45, 35), (46, 29), (47, 32), (48, 36), (49, 31), (50, 27), (51, 42), (52, 26), (53, 31), (54, 38), (55, 18), (56, 37), (57, 26), (58, 22), (59, 41), (60, 35), (61, 14), (62, 53), (63, 30), (64, 42), (65, 37), (66, 52), (67, 28), (68, 32), (69, 27), (70, 17), (71, 32), (72, 34), (73, 12), (74, 45), (75, 33), (76, 32), (77, 42), (78, 6), (79, 34), (80, 42), (81, 31), (82, 22), (83, 52), (84, 37), (85, 6), (86, 43), (87, 29), (88, 23), (89, 25), (90, 14), (91, 22), (92, 37), (93, 29), (94, 11), (95, 46), (96, 45), (97, 31), (98, 23), (99, 42)], 1: [(0, 49), (2, 

In [84]:
graph1 = Graph(adjacency_list, doc_embedding, 8)
graph1.a_star_algorithm(8, 80)

{0: 36, 1: 25, 2: 29, 3: 28, 4: 32, 5: 22, 6: 23, 7: 23, 8: 2, 9: 35, 10: 36, 11: 29, 12: 32, 13: 34, 14: 26, 15: 30, 16: 29, 17: 29, 18: 21, 19: 45, 20: 35, 21: 29, 22: 28, 23: 29, 24: 29, 25: 28, 26: 27, 27: 34, 28: 28, 29: 35, 30: 30, 31: 33, 32: 28, 33: 25, 34: 28, 35: 32, 36: 33, 37: 39, 38: 27, 39: 24, 40: 37, 41: 29, 42: 39, 43: 29, 44: 27, 45: 18, 46: 21, 47: 28, 48: 33, 49: 25, 50: 32, 51: 24, 52: 30, 53: 26, 54: 31, 55: 19, 56: 23, 57: 31, 58: 26, 59: 29, 60: 36, 61: 32, 62: 23, 63: 19, 64: 26, 65: 29, 66: 26, 67: 18, 68: 28, 69: 25, 70: 47, 71: 33, 72: 32, 73: 31, 74: 43, 75: 45, 76: 49, 77: 49, 78: 40, 79: 36, 80: 41, 81: 50, 82: 45, 83: 51, 84: 43, 85: 37, 86: 49, 87: 43, 88: 50, 89: 47, 90: 41, 91: 43, 92: 51, 93: 41, 94: 42, 95: 43, 96: 48, 97: 43, 98: 47, 99: 45}
Path found: [8, 70, 71, 73, 80]


[8, 70, 71, 73, 80]

In [55]:
c = 0
for se in sentences:
  print('doc {}: {}'.format(c, se))
  c +=1

doc 0: HURRYING through the tunnel at Veterans Stadium, the Los Angeles Rams' left guard, Tom Newberry, yelled, ''Warm-weather California sissies, huh?'' With a grin he disappeared into the locker room to calmly celebrate a 21-7 wild-card victory over the Philadelphia Eagles in a cold raw rain, which qualified the Rams to go against the Giants next Sunday at Giants Stadium in the Super Bowl XXIV tournament. But until yesterday, the Rams had always played like warm-weather California sissies in cold-weather playoff games. Until yesterday, the Rams were 0-9 in cold-weather playoff games since the franchise moved to Los Angeles from Cleveland more than four decades ago. In that time, Ram teams lost three outdoor playoff games in Minnesota (plus one in the warmth of the Metrodome a year ago), two in Washington, and one each in Cleveland, Detroit, Green Bay and Chicago. ''I'm tired of being a weatherman; if I get fired, I'm going right to the news,'' John Robinson, the Rams' coach, joked. '